In [62]:
import ccxt
import numpy as np
import pandas as pd
import talib
from datetime import datetime
import matplotlib.pyplot as plt

In [63]:
# first we need to have a list of ohlcv data
# let's get 1000 bars of 1 hour data from 2024-01-01 00:00:00 UTC

exchange = ccxt.binance()

symbol = 'BTC/USDT'
timeframe = '1h'
limit = 1000
since = exchange.parse8601('2024-01-01T00:00:00Z')

ohlcv = exchange.fetch_ohlcv(symbol, timeframe, since, limit)
df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
# print(df)

In [92]:
# Create numpy arrays of prices from the ohlcv data
# 0 : timestamp   1 : open   2 : high   3 : low   4 : close   5 : volume

Open = [candle[1] for candle in ohlcv]
Open = np.array(Open)

High = [candle[2] for candle in ohlcv]
High = np.array(High)

Low = [candle[3] for candle in ohlcv]
Low = np.array(Low)

Close = [candle[4] for candle in ohlcv]
Close = np.array(Close)

Volume = [candle[5] for candle in ohlcv]
Volume = np.array(Volume)

In [ ]:
# Some MAs. Note that MA50 needs 50 data points to start giving results, etc.

sma_20 = talib.SMA(Close, timeperiod=20)
sma_50 = talib.SMA(Close, timeperiod=50)
sma_200 = talib.SMA(Close, timeperiod=200)
# print(sma_20)

ema_20 = talib.EMA(Close, timeperiod=20)
ema_50 = talib.EMA(Close, timeperiod=50)
ema_200 = talib.EMA(Close, timeperiod=200)

wma_20 = talib.WMA(Close, timeperiod=20)
wma_50 = talib.WMA(Close, timeperiod=50)
wma_200 = talib.WMA(Close, timeperiod=200)

trima_20 = talib.TRIMA(Close, timeperiod=20)
trima_50 = talib.TRIMA(Close, timeperiod=50)
trima_200 = talib.TRIMA(Close, timeperiod=200)

In [ ]:
# plot the MAs. Ignore the first 200 data points because of the 200-period MA has 200 NaNs.

days = np.arange(limit)  

plt.figure(figsize=(10, 6))  
plt.plot(days, sma_20, label='SMA 20', color='blue')  
plt.plot(days, sma_50, label='SMA 50', color='orange')  
plt.plot(days, sma_200, label='SMA 200', color='green')  

plt.title('SMA 20, 50, 200')  
plt.xlabel('Days')  
plt.ylabel('Price')  
plt.legend()  
plt.grid(True)  

plt.show()

In [ ]:
# RSI(Relative Strength Index) ranges from 0 to 100.

rsi_14 = talib.RSI(Close, timeperiod=14)

rsi_7 = talib.RSI(Close, timeperiod=7)
rsi_21 = talib.RSI(Close, timeperiod=21)

# plot the RSI with only 100 data points
plt.figure(figsize=(10, 6))
plt.plot(days[25:125], rsi_14[25:125], label='RSI 14', color='blue')
plt.plot(days[25:125], rsi_7[25:125], label='RSI 7', color='orange')
plt.plot(days[25:125], rsi_21[25:125], label='RSI 21', color='green')
plt.title('RSI 7, 14, 21')
plt.xlabel('Days')
plt.ylabel('RSI')
plt.legend()
plt.grid(True)

plt.show()

In [ ]:
# MACD(Moving Average Convergence Divergence)

macd, macdsignal, macdhist = talib.MACD(Close, fastperiod=12, slowperiod=26, signalperiod=9)
DIF = macd # fast line
DEA = macdsignal # slow line aka signal line
HIST = macdhist

# plot the MACD with only 200 data points
plt.figure(figsize=(10, 6))
plt.plot(days[25:225], DIF[25:225], label='DIF', color='blue')
plt.plot(days[25:225], DEA[25:225], label='DEA', color='orange')
plt.bar(days[25:225], HIST[25:225], label='HIST', color='green')
plt.title('MACD')
plt.xlabel('Days')
plt.ylabel('Price')
plt.legend()
plt.grid(True)

plt.show()


In [ ]:
# Bollinger Bands

# nbdevup and nbdevdn are the number of standard deviations from the middle band
# matype is the type of moving average to be used. 0 stans for SMA, 1 for EMA, 2 for WMA, etc.
upperband, middleband, lowerband = talib.BBANDS(Close, timeperiod=20, nbdevup=2, nbdevdn=2, matype=0)

# plot the Bollinger Bands with only 300 data points
plt.figure(figsize=(10, 6))
plt.plot(days[125:425], Close[125:425], label='Close', color='black')
plt.plot(days[125:425], upperband[125:425], label='Upper Band', color='red')
plt.plot(days[125:425], middleband[125:425], label='Middle Band', color='blue')
plt.plot(days[125:425], lowerband[125:425], label='Lower Band', color='green')
plt.title('Bollinger Bands')
plt.xlabel('Days')
plt.ylabel('Price')
plt.legend()
plt.grid(True)

plt.show()

In [ ]:
# Stochastic Oscillator(隨機震盪指標), ranges from 0 to 100, 
# Indicates the relative position of the closing price between the highest and lowest prices of a period of time.
# >80 <20 are overbought and oversold levels.
# golden cross and death cross can be considered
# Bullish Divergence and Bearish Divergence can be considered

slowk, slowd = talib.STOCH(High, Low, Close, fastk_period=14, slowk_period=3, slowk_matype=0, slowd_period=3, slowd_matype=0)
# fastk_period is the range of the highest and lowest prices in. 

# plot the Stochastic Oscillator with only 100 data points
plt.figure(figsize=(10, 6))
plt.plot(days[225:325], slowk[225:325], label='Slow K', color='blue')
plt.plot(days[225:325], slowd[225:325], label='Slow D', color='orange')
plt.title('Stochastic Oscillator')
plt.xlabel('Days')
plt.ylabel('Price')
plt.legend()
plt.grid(True)

plt.show()

In [ ]:
# OBV(On Balance Volume) is a momentum indicator that uses volume flow to predict changes in stock price.
# OBV_t = OBV_t-1 + V_t    if C_t > C_t-1
# OBV_t = OBV_t-1 - V_t    if C_t < C_t-1
# OBV_t = OBV_t-1          if C_t = C_t-1

obv = talib.OBV(Close, Volume)

# plot the OBV with only 200 data points
fig, ax1 = plt.subplots(figsize=(10, 6))

color = 'black'
ax1.set_xlabel('Days')
ax1.set_ylabel('Price', color=color)
ax1.plot(days[25:225], Close[25:225], label='Close', color=color)
ax1.tick_params(axis='y', labelcolor=color)
ax1.grid(True)

ax2 = ax1.twinx()  
color = 'orange'
ax2.set_ylabel('OBV', color=color)
ax2.plot(days[25:225], obv[25:225], label='OBV', color=color)
ax2.tick_params(axis='y', labelcolor=color)

plt.title('OBV and Close Price')
fig.tight_layout()  

plt.show()

In [ ]:
# Volume Weighted Average Price(VWAP) is the average price throughout the day.
# However, TA-Lib does not have a function for VWAP.

In [ ]:
# Momentum, rate of acceleration of price or volume.

mom_10 = talib.MOM(Close, timeperiod=10)
mom_20 = talib.MOM(Close, timeperiod=20)
mom_50 = talib.MOM(Close, timeperiod=50)

# plot the different Momentum with only 100 data points
plt.figure(figsize=(10, 6))
plt.plot(days[150:250], mom_10[150:250], label='MOM 10', color='blue')
plt.plot(days[150:250], mom_20[150:250], label='MOM 20', color='orange')
plt.plot(days[150:250], mom_50[150:250], label='MOM 50', color='green')
plt.title('Momentum')
plt.xlabel('Days')
plt.ylabel('Price')
plt.legend()
plt.grid(True)

plt.show()

# plot the Momentum_10 with price with only 100 data points
fig, ax1 = plt.subplots(figsize=(10, 6))

Z = np.zeros(100)
color = 'black'
ax1.set_xlabel('Days')
ax1.set_ylabel('Price', color=color)
ax1.plot(days[150:250], Close[150:250], label='Close', color=color)
ax1.tick_params(axis='y', labelcolor=color)
ax1.grid(True)

ax2 = ax1.twinx()  
color = 'orange'
ax2.set_ylabel('MOM 10', color=color)
ax2.plot(days[150:250], mom_10[150:250], label='MOM 10', color=color)
ax2.plot(days[150:250], Z, label='0', color='red')
ax2.tick_params(axis='y', labelcolor=color)

plt.title('Momentum and Close Price')
plt.legend()
fig.tight_layout()  

plt.show()